In [ ]:
import pickle 
import pandas as pd
import numpy as np
import keras
from sklearn.metrics import confusion_matrix, classification_report
import matplotlib.pyplot as plt
import itertools
import math
from imblearn.over_sampling import RandomOverSampler

#select GPU to run script
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="2"

In [ ]:
###################################
#create the combined model VGG FNN#
###################################

def MLP_CNN():
    #Input: [Image matrix (shape = (n, y, x, num_channels), Centroid matrix (shape = (n, 2))]
    #E.g.: [I_train, C_train]
    #Y (Y_train) is of shape (n, n_classes)
    
    input1 = keras.layers.Input(shape=(y_image, x_image, num_channels),name = 'image_input')

    vgg_output =  vgg_conv(input1)

    for layer in vgg_conv.layers[:-4]:
        layer.trainable = False

    # Check the trainable status of the individual layers
    for layer in vgg_conv.layers:
        print(layer, layer.trainable)

    x1 = keras.layers.Flatten()(vgg_output)
    x1 = keras.layers.Dense(1024, activation='relu', name='dense_1')(x1)
    x1 = keras.layers.Dropout(0.5, name='dropout')(x1)

    input2 = keras.layers.Input(shape=(2,), name='coordinate_input')
    x2 = keras.layers.Dense(4, activation='relu', name='dense_2')(input2)

    merged = keras.layers.merge([x1, x2], mode='concat')
    out = keras.layers.Dense(n_classes, activation='sigmoid', name='output')(merged)

    model = keras.models.Model(inputs=[input1, input2], outputs=out)

    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=1e-4),
                  metrics=['acc'])
    model.summary()
    return(model)

In [ ]:
def MLP_CNN_BLSTM():
    #Input: [Image matrix (shape = (n, seq_len, y, x, num_channels), Centroid matrix (shape = (n, seq_len, 2))]
    #E.g.: [I_train, C_train]
    #Y (Y_train) is of shape (n, seq_len, n_classes)
    
    input_cnn = keras.layers.Input(shape=(y_image, x_image, num_channels),name = 'image_input')

    vgg_output =  vgg_conv(input_cnn)

    for layer in vgg_conv.layers[:-4]:
        layer.trainable = False

    # Check the trainable status of the individual layers
    for layer in vgg_conv.layers:
        print(layer, layer.trainable)

    x1 = keras.layers.Flatten()(vgg_output)
    x1 = keras.layers.Dense(1024, activation='relu', name='dense_1')(x1)
    x1 = keras.layers.Dropout(0.5, name='dropout')(x1)
    
    input_ffnn = keras.layers.Input(shape=(2,), name='coordinate_input')
    x2 = keras.layers.Dense(4, activation='relu', name='dense_2')(input_ffnn)

    ffnn = keras.models.Model(inputs=input_ffnn, outputs=x2)
    cnn = keras.models.Model(inputs=input_cnn, outputs=x1)

    I_sequence = keras.layers.Input(shape=(None, y_image, x_image, num_channels))
    X_sequence = keras.layers.Input(shape=(None, 2))

    time_x1 = keras.layers.TimeDistributed(ffnn)(X_sequence)
    time_x2 = keras.layers.TimeDistributed(cnn)(I_sequence)

    merged = keras.layers.merge([time_x1, time_x2], mode='concat')
    
    seq_dat = keras.layers.Bidirectional(keras.layers.LSTM(256, return_sequences=True))(merged)

    hidden = keras.layers.Dense(output_dim=1024, activation="relu")(seq_dat)
    outputs = keras.layers.Dense(output_dim=n_classes, activation="softmax")(hidden)
    
    model = keras.models.Model([I_sequence,X_sequence], outputs=outputs)
    
    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(lr=1e-4),
                  metrics=['acc'])
    model.summary()
    return(model)